<a href="https://colab.research.google.com/github/running-machin/cultural_analysis/blob/main/Topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyLDAvis

     |████████████████████████████████| 1.7MB 4.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 43.1MB/s 
     |████████████████████████████████| 15.3MB 268kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=170abe12950b4c321aed7a8da348e56aaad73fc7ae4a2f8d47febcd88f0a3407
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [1]:
# import necessary tools
import pandas as pd
import numpy as np


# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# create a stopwords list
stopwords = nltk.corpus.stopwords.words('english') #are lowercase
nltkStopwordsWithAdditions = stopwords + \
                             ['la', #comes up a lot (like 'la-la-la') but isn't a word so don't include it
                              'eh', 'ayy', #same as 'la'. 'eh' as in pronounced like 'aye'
                              'oh', 'ooh', #an exclamation with no real meaning but ranks highly so exclude
                              'us', 'u', 'ya', #different forms of 'you' which is a stopword
                              'na', 'ta', #alt forms of 'to' which is a stopword. From 'gonna', 'wanna', 'gotta'
                              'nah', #usage a mix of a sung-only word and slang for 'no' which is a stopword
                              'po', 'woo', 'doo', 'ah', 'uh', 'uhh' #sung-only words
                             ]

In [3]:
# add data from csv file
lyrics_path = "/content/drive/MyDrive/IRTM/kendrick_lamar_lyrics_with_filtered.csv"

In [4]:
# load the data
data = pd.read_csv(lyrics_path)
data.head(10)

,name,lyrics,album,year,album_name,pre-processed_lyrics
0,HUMBLE.,nobody pray for me it been that day for me wa...,"{'api_path': '/albums/337082', 'cover_art_url'...",2017,DAMN.,[Intro]\nNobody pray for me\nIt been that day ...
1,​m.A.A.d city,if pirus and crips all got along theyd probab...,"{'api_path': '/albums/491200', 'cover_art_url'...",2012,"good kid, m.A.A.d city (Deluxe Version)",[Part I: Produced by Sounwave]\n[Intro: Kendri...
2,Swimming Pools (Drank),pour up pass out now i done grew up round som...,"{'api_path': '/albums/491200', 'cover_art_url'...",2012,"good kid, m.A.A.d city (Deluxe Version)",[Produced by T-Minus]\n[Intro]\nPour up (Drank...
3,DNA.,i got i got i got i got loyalty got royalty i...,"{'api_path': '/albums/337082', 'cover_art_url'...",2017,DAMN.,"[Verse 1]\nI got, I got, I got, I got—\nLoyalt..."
4,Money Trees,uh me and my niggas tryna get it ya bish home...,"{'api_path': '/albums/491200', 'cover_art_url'...",2012,"good kid, m.A.A.d city (Deluxe Version)",[Produced by DJ Dahi]\n[Verse 1: Kendrick Lama...
5,XXX.,america god bless you if its good to ya ameri...,"{'api_path': '/albums/337082', 'cover_art_url'...",2017,DAMN.,[Intro: Bēkon & Kid Capri]\nAmerica\nGod bless...
6,"Bitch, Don’t Kill My Vibe",i am a sinner whos probably gonna sin again l...,"{'api_path': '/albums/491200', 'cover_art_url'...",2012,"good kid, m.A.A.d city (Deluxe Version)",[Produced by Sounwave]\n[Chorus: Kendrick Lama...
7,Poetic Justice,every second every minute man i swear that sh...,"{'api_path': '/albums/491200', 'cover_art_url'...",2012,"good kid, m.A.A.d city (Deluxe Version)",[Produced by Scoop DeVille & Sounwave]\n[Intro...
8,King Kunta,i got a bone to pick i dont want you monkey m...,"{'api_path': '/albums/120991', 'cover_art_url'...",2015,To Pimp a Butterfly,[Produced by Sounwave; Additional production b...
9,LOVE.,damn love or lust damn all of us give me a ru...,"{'api_path': '/albums/337082', 'cover_art_url'...",2017,DAMN.,"[Intro: Kendrick Lamar]\nDamn, love or lust\nD..."


In [5]:
tokenizer = RegexpTokenizer(r'\w+')
i=[]
for ind in range(len(data['lyrics'])):
  i.append(tokenizer.tokenize((str(data['lyrics'].loc[ind]))))
data["tokens"]=i
print(i)

[['nobody', 'pray', 'for', 'me', 'it', 'been', 'that', 'day', 'for', 'me', 'way', 'ayy', 'i', 'remember', 'syrup', 'sandwiches', 'and', 'crime', 'allowances', 'finesse', 'a', 'nigga', 'with', 'some', 'counterfeits', 'but', 'now', 'im', 'countin', 'this', 'parmesan', 'where', 'my', 'accountant', 'lives', 'in', 'fact', 'im', 'downin', 'this', 'd', 'ussé', 'with', 'my', 'boo', 'bae', 'tastes', 'like', 'kool', 'aid', 'for', 'the', 'analysts', 'girl', 'i', 'can', 'buy', 'your', 'ass', 'the', 'world', 'with', 'my', 'paystub', 'ooh', 'that', 'pussy', 'good', 'wont', 'you', 'sit', 'it', 'on', 'my', 'taste', 'bloods', 'i', 'get', 'way', 'too', 'petty', 'once', 'you', 'let', 'me', 'do', 'the', 'extras', 'pull', 'up', 'on', 'your', 'block', 'then', 'break', 'it', 'down', 'we', 'playin', 'tetris', 'am', 'to', 'the', 'pm', 'pm', 'to', 'the', 'am', 'funk', 'piss', 'out', 'your', 'per', 'diem', 'you', 'just', 'gotta', 'hate', 'em', 'funk', 'if', 'i', 'quit', 'your', 'bm', 'i', 'still', 'ride', 'merce

In [6]:
all_words = [word for tokens in data['tokens'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['tokens']]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

88512 words total, with a vocabulary size of 8180
Max sentence length is 2244


In [7]:
# import necessary tools
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [8]:
stopwords = nltk.corpus.stopwords.words('english')
words=['thats', 'yeah', 'got', 'get', 'im', 'its', 'dont']
for x in words:
    stopwords.append(x)
    
tf_vectorizer = CountVectorizer(stop_words=stopwords)
tf = tf_vectorizer.fit_transform(all_words)
tf_words = tf_vectorizer.get_feature_names()

In [9]:
# create an LDA model
ldamodel = LatentDirichletAllocation(n_components=6, max_iter=100, learning_method='online', 
                                     learning_offset=50., random_state=0)

In [10]:
# fit the model
ldamodel.fit(tf)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=100,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [11]:
ldamodel.score(tf)

-378772.8184604869

In [12]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(ldamodel, tf, tf_vectorizer, mds='tsne')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  error = np.finfo(np.float).max
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3       1.465019 -47.625065       1        1  16.717591
4      54.854797 -36.476795       2        1  16.684615
0      32.212929 -61.444572       3        1  16.675827
2       5.111096 -14.110332       4        1  16.659339
1      26.350170 -33.374702       5        1  16.635161
5      38.109818  -7.222781       6        1  16.627467, topic_info=        Term        Freq       Total Category  logprob  loglift
4035    like  704.000000  704.000000  Default  30.0000  30.0000
3871    know  423.000000  423.000000  Default  29.0000  29.0000
4709   nigga  358.000000  358.000000  Default  28.0000  28.0000
6273    shit  257.000000  257.000000  Default  27.0000  27.0000
229     aint  253.000000  253.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
4076  little   34.176060   34.884217   Topic6  -5.3997   1.7736
3276   heart   32.575058   33.283215   Topic6  -5.4477   1.7726
3480    hurt   32.134987   32.843145   Topic6  -5.4613   1.7723
7876    word   31.401401   32.109559   Topic6  -5.4844   1.7718
7111   thing   31.327209   32.035367   Topic6  -5.4868   1.7718

[210 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
229       1  0.998000     aint
282       5  0.991681  alright
401       6  0.995421   around
424       5  0.986310      ask
433       3  0.992898      ass
...     ...       ...      ...
7932      5  0.995454     yall
7954      4  0.980361      yes
7964      4  0.989722       yo
7971      1  0.973267    young
7976      3  0.995779    youre

[180 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 3, 2, 6])

In [ ]:
pyLDAvis.save_html(pyLDAvis.sklearn.prepare(ldamodel, tf, tf_vectorizer, mds='tsne'), './Lyrics_LDA_k_'+ "BoW" +'.html')

In [14]:
pip install langdetect

     |████████████████████████████████| 983kB 4.1MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=3e9eab089e2221c496cd1f25257b088e6c02cf599fb33e7f8e707073245e0e31
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect


In [29]:
import warnings
import csv
import os
import nltk
import pickle
from tqdm import tqdm
warnings.filterwarnings('ignore')
# execute the following of not downloaded already:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from langdetect import detect_langs
from langdetect.lang_detect_exception import LangDetectException

mainlyric_corpus = []
lang_filtered = 0
for filename in tqdm(os.listdir('/content/drive/MyDrive/IRTM')):
    # path = '/content/azlyrics-scraper/'+filename
    path = '/content/drive/MyDrive/IRTM/kendrick_lamar_lyrics.csv'
    with open(path, 'r') as f:
        data = csv.reader(f)
        headers = next(data) #skip headers
        for row in data:
            if len(row)< 2: #lyrics column not included
                continue
            if not row[1]: #lyrics are empty
                continue
            try:
                langlist = detect_langs(row[1])
                for l in langlist:
                    if l.prob < 0.95 or l.lang != 'en': #lyrics not clearly identified as English
                        lang_filtered += 1
                        continue
                    else:
                        mainlyric_corpus.append(row[1])
            except LangDetectException:
                continue

100%|██████████| 3/3 [00:11<00:00,  3.75s/it]


In [18]:
print("Filtered", lang_filtered, "songs that had lyrics identified in a language other than English.", len(mainlyric_corpus), "songs remain for topic modeling")

Filtered 19 songs that had lyrics identified in a language other than English. 760 songs remain for topic modeling


In [19]:
lyric_corpus = mainlyric_corpus

In [20]:
from nltk.tokenize import RegexpTokenizer

lyric_corpus_tokenized = []
tokenizer = RegexpTokenizer(r'\w+')
for lyric in tqdm(lyric_corpus):
    tokenized_lyric = tokenizer.tokenize(lyric.lower())  #tokenize and lower each lyric
    lyric_corpus_tokenized.append(tokenized_lyric)

100%|██████████| 760/760 [00:00<00:00, 6593.66it/s]


In [21]:
def count_token_in_corpus (corpus): #returns number of tokens for any given corpus
    return_count = 0
    for song in corpus:
        return_count += len(song)
    print("Total of", return_count, "tokens in the current corpus.")

In [22]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 434470 tokens in the current corpus.


In [23]:
len (lyric_corpus_tokenized)

760

In [24]:
for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    filtered_song = []    
    for token in song:
        if len(token) > 2 and not token.isnumeric():
            filtered_song.append(token)
    lyric_corpus_tokenized[s] = filtered_song

760it [00:00, 6409.43it/s]


In [25]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 327386 tokens in the current corpus.


In [30]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    lemmatized_tokens = []
    for token in song:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))
    lyric_corpus_tokenized[s] = lemmatized_tokens

760it [00:03, 229.73it/s]


In [31]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 327386 tokens in the current corpus.


In [32]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_stop_words = ['ooh','yeah','hey','whoa','woah', 'ohh', 'was', 'mmm', 'oooh','yah','yeh','mmm', 'hmm','deh','doh','jah','wa']
stop_words.extend(new_stop_words)

for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    filtered_text = []    
    for token in song:
        if token not in stop_words:
            filtered_text.append(token)
    lyric_corpus_tokenized[s] = filtered_text

760it [00:00, 837.52it/s]


In [33]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 220896 tokens in the current corpus.


In [34]:
!wget https://www.freewebheaders.com/download/files/youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.zip

--2021-05-28 14:35:24--  https://www.freewebheaders.com/download/files/youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.zip
Resolving www.freewebheaders.com (www.freewebheaders.com)... 172.67.201.102, 104.21.36.247, 2606:4700:3032::ac43:c966, ...
Connecting to www.freewebheaders.com (www.freewebheaders.com)|172.67.201.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6684 (6.5K) [application/zip]
Saving to: ‘youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.zip.1’

youtube-blacklist-w 100%[===================>]   6.53K  --.-KB/s    in 0s      

2021-05-28 14:35:24 (43.9 MB/s) - ‘youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.zip.1’ saved [6684/6684]



In [35]:
!unzip /content/youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.zip

Archive:  /content/youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.zip
replace youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: youtube-blacklist-words-list_comma-separated-text-file_2021-01-19.txt  


In [39]:
profanities_raw = """2 girls 1 cup, 2g1c, 4r5e, 5h1t, 5hit, a$$, a$$hole, a_s_s, a2m, a54, a55, a55hole, aeolus, ahole, alabama hot pocket, alaskan pipeline, anal, anal impaler, anal leakage, analannie, analprobe, analsex, anilingus, anus, apeshit, ar5e, areola, areole, arian, arrse, arse, arsehole, aryan, ass, ass fuck, ass hole, assault, assbag, assbagger, assbandit, assbang, assbanged, assbanger, assbangs, assbite, assblaster, assclown, asscock, asscracker, asses, assface, assfaces, assfuck, assfucker, ass-fucker, assfukka, assgoblin, assh0le, asshat, ass-hat, asshead, assho1e, asshole, assholes, asshopper, asshore, ass-jabber, assjacker, assjockey, asskiss, asskisser, assklown, asslick, asslicker, asslover, assman, assmaster, assmonkey, assmucus, assmunch, assmuncher, assnigger, asspacker, asspirate, ass-pirate, asspuppies, assranger, assshit, assshole, asssucker, asswad, asswhole, asswhore, asswipe, asswipes, auto erotic, autoerotic, axwound, azazel, azz, b!tch, b00bs, b17ch, b1tch, babe, babeland, babes, baby batter, baby juice, badfuck, ball gag, ball gravy, ball kicking, ball licking, ball sack, ball sucking, ballbag, balllicker, balls, ballsack, bampot, bang, bang (one's) box, bangbros, banger, banging, bareback, barely legal, barenaked, barf, barface, barfface, bastard, bastardo, bastards, bastinado, batty boy, bawdy, bazongas, bazooms, bbw, bdsm, beaner, beaners, beardedclam, beastial, beastiality, beatch, beater, beatyourmeat, beaver, beaver cleaver, beaver lips, beef curtain, beef curtains, beer, beeyotch, bellend, bender, beotch, bestial, bestiality, bi+ch, biatch, bicurious, big black, big breasts, big knockers, big tits, bigbastard, bigbutt, bigger, bigtits, bimbo, bimbos, bint, birdlock, bisexual, bi-sexual, bitch, bitch tit, bitchass, bitched, bitcher, bitchers, bitches, bitchez, bitchin, bitching, bitchtits, bitchy, black cock, blonde action, blonde on blonde action, bloodclaat, bloody, bloody hell, blow, blow job, blow me, blow mud, blow your load, blowjob, blowjobs, blue waffle, blumpkin, boang, bod, bodily, bogan, bohunk, boink, boiolas, bollick, bollock, bollocks, bollok, bollox, bomd, bondage, bone, boned, boner, boners, bong, boob, boobies, boobs, booby, booger, bookie, boong, boonga, booobs, boooobs, booooobs, booooooobs, bootee, bootie, booty, booty call, booze, boozer, boozy, bosom, bosomy, bowel, bowels, bra, brassiere, breast, breastjob, breastlover, breastman, breasts, breeder, brotherfucker, brown showers, brunette action, buceta, bugger, buggered, buggery, bukkake, bull shit, bullcrap, bulldike, bulldyke, bullet vibe, bullshit, bullshits, bullshitted, bullturds, bum, bum boy, bumblefuck, bumclat, bumfuck, bummer, bung, bung hole, bunga, bunghole, bunny fucker, bust a load, busty, butchdike, butchdyke, butt, butt fuck, butt plug, buttbang, butt-bang, buttcheeks, buttface, buttfuck, butt-fuck, buttfucka, buttfucker, butt-fucker, butthead, butthole, buttman, buttmuch, buttmunch, buttmuncher, butt-pirate, buttplug, c.0.c.k, c.o.c.k., c.u.n.t, c0ck, c-0-c-k, c0cksucker, caca, cahone, camel toe, cameltoe, camgirl, camslut, camwhore, carpet muncher, carpetmuncher, cawk, cervix, chesticle, chi-chi man, chick with a dick, child-fucker, chin, chinc, chincs, chink, chinky, choad, choade, choc ice, chocolate rosebuds, chode, chodes, chota bags, cipa, circlejerk, cl1t, cleveland steamer, climax, clit, clit licker, clitface, clitfuck, clitoris, clitorus, clits, clitty, clitty litter, clogwog, clover clamps, clunge, clusterfuck, cnut, cocain, cocaine, cock, c-o-c-k, cock pocket, cock snot, cock sucker, cockass, cockbite, cockblock, cockburger, cockeye, cockface, cockfucker, cockhead, cockholster, cockjockey, cockknocker, cockknoker, cocklicker, cocklover, cocklump, cockmaster, cockmongler, cockmongruel, cockmonkey, cockmunch, cockmuncher, cocknose, cocknugget, cocks, cockshit, cocksmith, cocksmoke, cocksmoker, cocksniffer, cocksucer, cocksuck, cocksuck , cocksucked, cocksucker, cock-sucker, cocksuckers, cocksucking, cocksucks, cocksuka, cocksukka, cockwaffle, coffin dodger, coital, cok, cokmuncher, coksucka, commie, condom, coochie, coochy, coon, coonnass, coons, cooter, cop some wood, coprolagnia, coprophilia, corksucker, cornhole, corp whore, cox, crabs, crack, cracker, crackwhore, crack-whore, crap, crappy, creampie, cretin, crikey, cripple, crotte, cum, cum chugger, cum dumpster, cum freak, cum guzzler, cumbubble, cumdump, cumdumpster, cumguzzler, cumjockey, cummer, cummin, cumming, cums, cumshot, cumshots, cumslut, cumstain, cumtart, cunilingus, cunillingus, cunn, cunnie, cunnilingus, cunntt, cunny, cunt, c-u-n-t, cunt hair, cuntass, cuntbag, cuntface, cuntfuck, cuntfucker, cunthole, cunthunter, cuntlick, cuntlick , cuntlicker, cuntlicker , cuntlicking, cuntrag, cunts, cuntsicle, cuntslut, cunt-struck, cuntsucker, cut rope, cyalis, cyberfuc, cyberfuck, cyberfucked, cyberfucker, cyberfuckers, cyberfucking, cybersex, d0ng, d0uch3, d0uche, d1ck, d1ld0, d1ldo, dago, dagos, dammit, damn, damned, damnit, darkie, darn, date rape, daterape, dawgie-style, deep throat, deepthroat, deggo, dendrophilia, dick, dick head, dick hole, dick shy, dickbag, dickbeaters, dickbrain, dickdipper, dickface, dickflipper, dickfuck, dickfucker, dickhead, dickheads, dickhole, dickish, dick-ish, dickjuice, dickmilk, dickmonger, dickripper, dicks, dicksipper, dickslap, dick-sneeze, dicksucker, dicksucking, dicktickler, dickwad, dickweasel, dickweed, dickwhipper, dickwod, dickzipper, diddle, dike, dildo, dildos, diligaf, dillweed, dimwit, dingle, dingleberries, dingleberry, dink, dinks, dipship, dipshit, dirsa, dirty, dirty pillows, dirty sanchez, dlck, dog style, dog-fucker, doggie style, doggiestyle, doggie-style, doggin, dogging, doggy style, doggystyle, doggy-style, dolcett, domination, dominatrix, dommes, dong, donkey punch, donkeypunch, donkeyribber, doochbag, doofus, dookie, doosh, dopey, double dong, double penetration, doublelift, douch3, douche, douchebag, douchebags, douche-fag, douchewaffle, douchey, dp action, drunk, dry hump, duche, dumass, dumb ass, dumbass, dumbasses, dumbcunt, dumbfuck, dumbshit, dummy, dumshit, dvda, dyke, dykes, eat a dick, eat hair pie, eat my ass, eatpussy, ecchi, ejaculate, ejaculated, ejaculates, ejaculating, ejaculatings, ejaculation, ejakulate, enlargement, erect, erection, erotic, erotism, escort, essohbee, eunuch, extacy, extasy, f u c k, f u c k e r, f.u.c.k, f_u_c_k, f4nny, facefucker, facial, fack, fag, fagbag, fagfucker, fagg, fagged, fagging, faggit, faggitt, faggot, faggotcock, faggots, faggs, fagot, fagots, fags, fagtard, faig, faigt, fanny, fannybandit, fannyflaps, fannyfucker, fanyy, fart, fartknocker, fastfuck, fat, fatass, fatfuck, fatfucker, fcuk, fcuker, fcuking, fecal, feck, fecker, felch, felcher, felching, fellate, fellatio, feltch, feltcher, female squirting, femdom, fenian, figging, fingerbang, fingerfuck, fingerfuck , fingerfucked, fingerfucker, fingerfucker , fingerfuckers, fingerfucking, fingerfucks, fingering, fist fuck, fisted, fistfuck, fistfucked, fistfucker, fistfucker , fistfuckers, fistfucking, fistfuckings, fistfucks, fisting, fisty, flamer, flange, flaps, fleshflute, flog the log, floozy, foad, foah, fondle, foobar, fook, fooker, foot fetish, footfuck, footfucker, footjob, footlicker, foreskin, freakfuck, freakyfucker, freefuck, freex, frigg, frigga, frotting, fubar, fuc, fuck, f-u-c-k, fuck buttons, fuck hole, fuck off, fuck puppet, fuck trophy, fuck yo mama, fuck you, fucka, fuckass, fuck-ass, fuckbag, fuck-bitch, fuckboy, fuckbrain, fuckbutt, fuckbutter, fucked, fuckedup, fucker, fuckers, fuckersucker, fuckface, fuckfreak, fuckhead, fuckheads, fuckher, fuckhole, fuckin, fucking, fuckingbitch, fuckings, fuckingshitmotherfucker, fuckme, fuckme , fuckmeat, fuckmehard, fuckmonkey, fucknugget, fucknut, fucknutt, fuckoff, fucks, fuckstick, fucktard, fuck-tard, fucktards, fucktart, fucktoy, fucktwat, fuckup, fuckwad, fuckwhit, fuckwhore, fuckwit, fuckwitt, fuckyou, fudge packer, fudgepacker, fudge-packer, fuk, fuker, fukker, fukkers, fukkin, fuks, fukwhit, fukwit, fuq, futanari, fux, fux0r, fvck, fxck, gae, gai, gang bang, gangbang, gang-bang, gangbanged, gangbangs, ganja, gash, gassy ass, gay sex, gayass, gaybob, gaydo, gayfuck, gayfuckist, gaylord, gays, gaysex, gaytard, gaywad, gender bender, genitals, gey, gfy, ghay, ghey, giant cock, gigolo, ginger, gippo, girl on, girl on top, girls gone wild, git, glans, goatcx, goatse, god damn, godamn, godamnit, goddam, god-dam, goddammit, goddamn, goddamned, god-damned, goddamnit, goddamnmuthafucker, godsdamn, gokkun, golden shower, goldenshower, golliwog, gonad, gonads, gonorrehea, goo girl, gooch, goodpoop, gook, gooks, goregasm, gotohell, gringo, grope, group sex, gspot, g-spot, gtfo, guido, guro, h0m0, h0mo, ham flap, hand job, handjob, hard core, hard on, hardcore, hardcoresex, he11, headfuck, hebe, heeb, hell, hemp, hentai, heroin, herp, herpes, herpy, heshe, he-she, hitler, hiv, ho, hoar, hoare, hobag, hoe, hoer, holy shit, hom0, homey, homo, homodumbshit, homoerotic, homoey, honkey, honky, hooch, hookah, hooker, hoor, hootch, hooter, hooters, hore, horniest, horny, hot carl, hot chick, hotpussy, hotsex, how to kill, how to murdep, how to murder, huge fat, hump, humped, humping, hun, hussy, hymen, iap, iberian slap, inbred, incest, injun, intercourse, j3rk0ff, jack off, jackass, jackasses, jackhole, jackoff, jack-off, jaggi, jagoff, jail bait, jailbait, jap, japs, jelly donut, jerk, jerk off, jerk0ff, jerkass, jerked, jerkoff, jerk-off, jigaboo, jiggaboo, jiggerboo, jism, jiz, jizm, jizz, jizzed, jock, juggs, jungle bunny, junglebunny, junkie, junky, kafir, kawk, kike, kikes, kill, kinbaku, kinkster, kinky, kkk, klan, knob, knob end, knobbing, knobead, knobed, knobend, knobhead, knobjocky, knobjokey, kock, kondum, kondums, kooch, kooches, kootch, kraut, kum, kummer, kumming, kums, kunilingus, kunja, kunt, kwif, kyke, l3i+ch, l3itch, labia, lameass, lardass, leather restraint, leather straight jacket, lech, lemon party, leper, lesbian, lesbians, lesbo, lesbos, lez, lezbian, lezbians, lezbo, lezbos, lezza, lezzie, lezzies, lezzy, lmao, lmfao, loin, loins, lolita, looney, lovemaking, lube, lust, lusting, lusty, m0f0, m0fo, m45terbate, ma5terb8, ma5terbate, mafugly, make me come, male squirting, mams, masochist, massa, masterb8, masterbat, masterbat3, masterbate, master-bate, masterbating, masterbation, masterbations, masturbate, masturbating, masturbation, maxi, mcfagget, menage a trois, menses, menstruate, menstruation, meth, m-fucking, mick, middle finger, midget, milf, minge, minger, missionary position, mof0, mofo, mo-fo, molest, mong, moo moo foo foo, moolie, moron, mothafuck, mothafucka, mothafuckas, mothafuckaz, mothafucked, mothafucker, mothafuckers, mothafuckin, mothafucking, mothafuckings, mothafucks, mother fucker, motherfuck, motherfucka, motherfucked, motherfucker, motherfuckers, motherfuckin, motherfucking, motherfuckings, motherfuckka, motherfucks, mound of venus, mr hands, mtherfucker, mthrfucker, mthrfucking, muff, muff diver, muff puff, muffdiver, muffdiving, munging, munter, murder, mutha, muthafecker, muthafuckaz, muthafuckker, muther, mutherfucker, mutherfucking, muthrfucking, n1gga, n1gger, nad, nads, naked, nambla, napalm, nappy, nawashi, nazi, nazism, need the dick, negro, neonazi, nig nog, nigaboo, nigg3r, nigg4h, nigga, niggah, niggas, niggaz, nigger, niggers, niggle, niglet, nig-nog, nimphomania, nimrod, ninny, nipple, nipples, nob, nob jokey, nobhead, nobjocky, nobjokey, nonce, nooky, nsfw images, nude, nudity, numbnuts, nut butter, nut sack, nutsack, nutter, nympho, nymphomania, octopussy, old bag, omg, omorashi, one cup two girls, one guy one jar, opiate, opium, oral, orally, organ, orgasim, orgasims, orgasm, orgasmic, orgasms, orgies, orgy, ovary, ovum, ovums, p.u.s.s.y., p0rn, paddy, paedophile, paki, panooch, pansy, pantie, panties, panty, pastie, pasty, pawn, pcp, pecker, peckerhead, pedo, pedobear, pedophile, pedophilia, pedophiliac, pee, peepee, pegging, penetrate, penetration, penial, penile, penis, penisbanger, penisfucker, penispuffer, perversion, peyote, phalli, phallic, phone sex, phonesex, phuck, phuk, phuked, phuking, phukked, phukking, phuks, phuq, piece of shit, pigfucker, pikey, pillowbiter, pimp, pimpis, pinko, piss, piss off, piss pig, pissed, pissed off, pisser, pissers, pisses, pissflaps, pissin, pissing, pissoff, piss-off, pisspig, playboy, pleasure chest, pms, polack, pole smoker, polesmoker, pollock, ponyplay, poof, poon, poonani, poonany, poontang, poop, poop chute, poopchute, poopuncher, porch monkey, porchmonkey, porn, porno, pornography, pornos, pot, potty, prick, pricks, prickteaser, prig, prince albert piercing, prod, pron, prostitute, prude, psycho, pthc, pube, pubes, pubic, pubis, punani, punanny, punany, punkass, punky, punta, puss, pusse, pussi, pussies, pussy, pussy fart, pussy palace, pussylicking, pussypounder, pussys, pust, puto, queaf, queef, queer, queerbait, queerhole, queero, queers, quicky, quim, racy, raghead, raging boner, rape, raped, raper, rapey, raping, rapist, raunch, rectal, rectum, rectus, reefer, reetard, reich, renob, retard, retarded, reverse cowgirl, revue, rimjaw, rimjob, rimming, ritard, rosy palm, rosy palm and her 5 sisters, rtard, r-tard, rubbish, rum, rump, rumprammer, ruski, rusty trombone, s hit, s&m, s.h.i.t., s.o.b., s_h_i_t, s0b, sadism, sadist, sambo, sand nigger, sandbar, sandler, sandnigger, sanger, santorum, sausage queen, scag, scantily, scat, schizo, schlong, scissoring, screw, screwed, screwing, scroat, scrog, scrot, scrote, scrotum, scrud, scum, seaman, seamen, seduce, seks, semen, sex, sexo, sexual, sexy, sh!+, sh!t, sh1t, s-h-1-t, shag, shagger, shaggin, shagging, shamedame, shaved beaver, shaved pussy, shemale, shi+, shibari, shirt lifter, shit, s-h-i-t, shit ass, shit fucker, shitass, shitbag, shitbagger, shitblimp, shitbrains, shitbreath, shitcanned, shitcunt, shitdick, shite, shiteater, shited, shitey, shitface, shitfaced, shitfuck, shitfull, shithead, shitheads, shithole, shithouse, shiting, shitings, shits, shitspitter, shitstain, shitt, shitted, shitter, shitters, shittier, shittiest, shitting, shittings, shitty, shiz, shiznit, shota, shrimping, sissy, skag, skank, skeet, skullfuck, slag, slanteye, slave, sleaze, sleazy, slope, slut, slut bucket, slutbag, slutdumper, slutkiss, sluts, smartass, smartasses, smeg, smegma, smut, smutty, snatch, sniper, snowballing, snuff, s-o-b, sod off, sodom, sodomize, sodomy, son of a bitch, son of a motherless goat, son of a whore, son-of-a-bitch, souse, soused, spac, spade, sperm, spic, spick, spik, spiks, splooge, splooge moose, spooge, spook, spread legs, spunk, steamy, stfu, stiffy, stoned, strap on, strapon, strappado, strip, strip club, stroke, stupid, style doggy, suck, suckass, sucked, sucking, sucks, suicide girls, sultry women, sumofabiatch, swastika, swinger, t1t, t1tt1e5, t1tties, taff, taig, tainted love, taking the piss, tampon, tard, tart, taste my, tawdry, tea bagging, teabagging, teat, teets, teez, terd, teste, testee, testes, testical, testicle, testis, threesome, throating, thrust, thug, thundercunt, tied up, tight white, tinkle, tit, tit wank, titfuck, titi, tities, tits, titt, tittie5, tittiefucker, titties, titty, tittyfuck, tittyfucker, tittywank, titwank, toke, tongue in a, toots, topless, tosser, towelhead, tramp, tranny, transsexual, trashy, tribadism, trumped, tub girl, tubgirl, turd, tush, tushy, tw4t, twat, twathead, twatlips, twats, twatty, twatwaffle, twink, twinkie, two fingers, two fingers with tongue, two girls one cup, twunt, twunter, ugly, unclefucker, undies, undressing, unwed, upskirt, urethra play, urinal, urine, urophilia, uterus, uzi, v14gra, v1gra, vag, vagina, vajayjay, va-j-j, valium, venus mound, veqtable, viagra, vibrator, violet wand, virgin, vixen, vjayjay, vodka, vomit, vorarephilia, voyeur, vulgar, vulva, w00se, wad, wang, wank, wanker, wankjob, wanky, wazoo, wedgie, weed, weenie, weewee, weiner, weirdo, wench, wet dream, wetback, wh0re, wh0reface, white power, whitey, whiz, whoar, whoralicious, whore, whorealicious, whorebag, whored, whoreface, whorehopper, whorehouse, whores, whoring, wigger, willies, willy, window licker, wiseass, wiseasses, wog, womb, woody, wop, wrapping men, wrinkled starfish, wtf, xrated, x-rated, xx, xxx, yaoi, yeasty, yellow showers, yid, yiffy, yobbo, zoophile, zoophilia2 girls 1 cup, 2g1c, 4r5e, 5h1t, 5hit, a$$, a$$hole, a_s_s, a2m, a54, a55, a55hole, aeolus, ahole, alabama hot pocket, alaskan pipeline, anal, anal impaler, anal leakage, analannie, analprobe, analsex, anilingus, anus, apeshit, ar5e, areola, areole, arian, arrse, arse, arsehole, aryan, ass, ass fuck, ass hole, assault, assbag, assbagger, assbandit, assbang, assbanged, assbanger, assbangs, assbite, assblaster, assclown, asscock, asscracker, asses, assface, assfaces, assfuck, assfucker, ass-fucker, assfukka, assgoblin, assh0le, asshat, ass-hat, asshead, assho1e, asshole, assholes, asshopper, asshore, ass-jabber, assjacker, assjockey, asskiss, asskisser, assklown, asslick, asslicker, asslover, assman, assmaster, assmonkey, assmucus, assmunch, assmuncher, assnigger, asspacker, asspirate, ass-pirate, asspuppies, assranger, assshit, assshole, asssucker, asswad, asswhole, asswhore, asswipe, asswipes, auto erotic, autoerotic, axwound, azazel, azz, b!tch, b00bs, b17ch, b1tch, babe, babeland, babes, baby batter, baby juice, badfuck, ball gag, ball gravy, ball kicking, ball licking, ball sack, ball sucking, ballbag, balllicker, balls, ballsack, bampot, bang, bang (one's) box, bangbros, banger, banging, bareback, barely legal, barenaked, barf, barface, barfface, bastard, bastardo, bastards, bastinado, batty boy, bawdy, bazongas, bazooms, bbw, bdsm, beaner, beaners, beardedclam, beastial, beastiality, beatch, beater, beatyourmeat, beaver, beaver cleaver, beaver lips, beef curtain, beef curtains, beer, beeyotch, bellend, bender, beotch, bestial, bestiality, bi+ch, biatch, bicurious, big black, big breasts, big knockers, big tits, bigbastard, bigbutt, bigger, bigtits, bimbo, bimbos, bint, birdlock, bisexual, bi-sexual, bitch, bitch tit, bitchass, bitched, bitcher, bitchers, bitches, bitchez, bitchin, bitching, bitchtits, bitchy, black cock, blonde action, blonde on blonde action, bloodclaat, bloody, bloody hell, blow, blow job, blow me, blow mud, blow your load, blowjob, blowjobs, blue waffle, blumpkin, boang, bod, bodily, bogan, bohunk, boink, boiolas, bollick, bollock, bollocks, bollok, bollox, bomd, bondage, bone, boned, boner, boners, bong, boob, boobies, boobs, booby, booger, bookie, boong, boonga, booobs, boooobs, booooobs, booooooobs, bootee, bootie, booty, booty call, booze, boozer, boozy, bosom, bosomy, bowel, bowels, bra, brassiere, breast, breastjob, breastlover, breastman, breasts, breeder, brotherfucker, brown showers, brunette action, buceta, bugger, buggered, buggery, bukkake, bull shit, bullcrap, bulldike, bulldyke, bullet vibe, bullshit, bullshits, bullshitted, bullturds, bum, bum boy, bumblefuck, bumclat, bumfuck, bummer, bung, bung hole, bunga, bunghole, bunny fucker, bust a load, busty, butchdike, butchdyke, butt, butt fuck, butt plug, buttbang, butt-bang, buttcheeks, buttface, buttfuck, butt-fuck, buttfucka, buttfucker, butt-fucker, butthead, butthole, buttman, buttmuch, buttmunch, buttmuncher, butt-pirate, buttplug, c.0.c.k, c.o.c.k., c.u.n.t, c0ck, c-0-c-k, c0cksucker, caca, cahone, camel toe, cameltoe, camgirl, camslut, camwhore, carpet muncher, carpetmuncher, cawk, cervix, chesticle, chi-chi man, chick with a dick, child-fucker, chin, chinc, chincs, chink, chinky, choad, choade, choc ice, chocolate rosebuds, chode, chodes, chota bags, cipa, circlejerk, cl1t, cleveland steamer, climax, clit, clit licker, clitface, clitfuck, clitoris, clitorus, clits, clitty, clitty litter, clogwog, clover clamps, clunge, clusterfuck, cnut, cocain, cocaine, cock, c-o-c-k, cock pocket, cock snot, cock sucker, cockass, cockbite, cockblock, cockburger, cockeye, cockface, cockfucker, cockhead, cockholster, cockjockey, cockknocker, cockknoker, cocklicker, cocklover, cocklump, cockmaster, cockmongler, cockmongruel, cockmonkey, cockmunch, cockmuncher, cocknose, cocknugget, cocks, cockshit, cocksmith, cocksmoke, cocksmoker, cocksniffer, cocksucer, cocksuck, cocksuck , cocksucked, cocksucker, cock-sucker, cocksuckers, cocksucking, cocksucks, cocksuka, cocksukka, cockwaffle, coffin dodger, coital, cok, cokmuncher, coksucka, commie, condom, coochie, coochy, coon, coonnass, coons, cooter, cop some wood, coprolagnia, coprophilia, corksucker, cornhole, corp whore, cox, crabs, crack, cracker, crackwhore, crack-whore, crap, crappy, creampie, cretin, crikey, cripple, crotte, cum, cum chugger, cum dumpster, cum freak, cum guzzler, cumbubble, cumdump, cumdumpster, cumguzzler, cumjockey, cummer, cummin, cumming, cums, cumshot, cumshots, cumslut, cumstain, cumtart, cunilingus, cunillingus, cunn, cunnie, cunnilingus, cunntt, cunny, cunt, c-u-n-t, cunt hair, cuntass, cuntbag, cuntface, cuntfuck, cuntfucker, cunthole, cunthunter, cuntlick, cuntlick , cuntlicker, cuntlicker , cuntlicking, cuntrag, cunts, cuntsicle, cuntslut, cunt-struck, cuntsucker, cut rope, cyalis, cyberfuc, cyberfuck, cyberfucked, cyberfucker, cyberfuckers, cyberfucking, cybersex, d0ng, d0uch3, d0uche, d1ck, d1ld0, d1ldo, dago, dagos, dammit, damn, damned, damnit, darkie, darn, date rape, daterape, dawgie-style, deep throat, deepthroat, deggo, dendrophilia, dick, dick head, dick hole, dick shy, dickbag, dickbeaters, dickbrain, dickdipper, dickface, dickflipper, dickfuck, dickfucker, dickhead, dickheads, dickhole, dickish, dick-ish, dickjuice, dickmilk, dickmonger, dickripper, dicks, dicksipper, dickslap, dick-sneeze, dicksucker, dicksucking, dicktickler, dickwad, dickweasel, dickweed, dickwhipper, dickwod, dickzipper, diddle, dike, dildo, dildos, diligaf, dillweed, dimwit, dingle, dingleberries, dingleberry, dink, dinks, dipship, dipshit, dirsa, dirty, dirty pillows, dirty sanchez, dlck, dog style, dog-fucker, doggie style, doggiestyle, doggie-style, doggin, dogging, doggy style, doggystyle, doggy-style, dolcett, domination, dominatrix, dommes, dong, donkey punch, donkeypunch, donkeyribber, doochbag, doofus, dookie, doosh, dopey, double dong, double penetration, doublelift, douch3, douche, douchebag, douchebags, douche-fag, douchewaffle, douchey, dp action, drunk, dry hump, duche, dumass, dumb ass, dumbass, dumbasses, dumbcunt, dumbfuck, dumbshit, dummy, dumshit, dvda, dyke, dykes, eat a dick, eat hair pie, eat my ass, eatpussy, ecchi, ejaculate, ejaculated, ejaculates, ejaculating, ejaculatings, ejaculation, ejakulate, enlargement, erect, erection, erotic, erotism, escort, essohbee, eunuch, extacy, extasy, f u c k, f u c k e r, f.u.c.k, f_u_c_k, f4nny, facefucker, facial, fack, fag, fagbag, fagfucker, fagg, fagged, fagging, faggit, faggitt, faggot, faggotcock, faggots, faggs, fagot, fagots, fags, fagtard, faig, faigt, fanny, fannybandit, fannyflaps, fannyfucker, fanyy, fart, fartknocker, fastfuck, fat, fatass, fatfuck, fatfucker, fcuk, fcuker, fcuking, fecal, feck, fecker, felch, felcher, felching, fellate, fellatio, feltch, feltcher, female squirting, femdom, fenian, figging, fingerbang, fingerfuck, fingerfuck , fingerfucked, fingerfucker, fingerfucker , fingerfuckers, fingerfucking, fingerfucks, fingering, fist fuck, fisted, fistfuck, fistfucked, fistfucker, fistfucker , fistfuckers, fistfucking, fistfuckings, fistfucks, fisting, fisty, flamer, flange, flaps, fleshflute, flog the log, floozy, foad, foah, fondle, foobar, fook, fooker, foot fetish, footfuck, footfucker, footjob, footlicker, foreskin, freakfuck, freakyfucker, freefuck, freex, frigg, frigga, frotting, fubar, fuc, fuck, f-u-c-k, fuck buttons, fuck hole, fuck off, fuck puppet, fuck trophy, fuck yo mama, fuck you, fucka, fuckass, fuck-ass, fuckbag, fuck-bitch, fuckboy, fuckbrain, fuckbutt, fuckbutter, fucked, fuckedup, fucker, fuckers, fuckersucker, fuckface, fuckfreak, fuckhead, fuckheads, fuckher, fuckhole, fuckin, fucking, fuckingbitch, fuckings, fuckingshitmotherfucker, fuckme, fuckme , fuckmeat, fuckmehard, fuckmonkey, fucknugget, fucknut, fucknutt, fuckoff, fucks, fuckstick, fucktard, fuck-tard, fucktards, fucktart, fucktoy, fucktwat, fuckup, fuckwad, fuckwhit, fuckwhore, fuckwit, fuckwitt, fuckyou, fudge packer, fudgepacker, fudge-packer, fuk, fuker, fukker, fukkers, fukkin, fuks, fukwhit, fukwit, fuq, futanari, fux, fux0r, fvck, fxck, gae, gai, gang bang, gangbang, gang-bang, gangbanged, gangbangs, ganja, gash, gassy ass, gay sex, gayass, gaybob, gaydo, gayfuck, gayfuckist, gaylord, gays, gaysex, gaytard, gaywad, gender bender, genitals, gey, gfy, ghay, ghey, giant cock, gigolo, ginger, gippo, girl on, girl on top, girls gone wild, git, glans, goatcx, goatse, god damn, godamn, godamnit, goddam, god-dam, goddammit, goddamn, goddamned, god-damned, goddamnit, goddamnmuthafucker, godsdamn, gokkun, golden shower, goldenshower, golliwog, gonad, gonads, gonorrehea, goo girl, gooch, goodpoop, gook, gooks, goregasm, gotohell, gringo, grope, group sex, gspot, g-spot, gtfo, guido, guro, h0m0, h0mo, ham flap, hand job, handjob, hard core, hard on, hardcore, hardcoresex, he11, headfuck, hebe, heeb, hell, hemp, hentai, heroin, herp, herpes, herpy, heshe, he-she, hitler, hiv, ho, hoar, hoare, hobag, hoe, hoer, holy shit, hom0, homey, homo, homodumbshit, homoerotic, homoey, honkey, honky, hooch, hookah, hooker, hoor, hootch, hooter, hooters, hore, horniest, horny, hot carl, hot chick, hotpussy, hotsex, how to kill, how to murdep, how to murder, huge fat, hump, humped, humping, hun, hussy, hymen, iap, iberian slap, inbred, incest, injun, intercourse, j3rk0ff, jack off, jackass, jackasses, jackhole, jackoff, jack-off, jaggi, jagoff, jail bait, jailbait, jap, japs, jelly donut, jerk, jerk off, jerk0ff, jerkass, jerked, jerkoff, jerk-off, jigaboo, jiggaboo, jiggerboo, jism, jiz, jizm, jizz, jizzed, jock, juggs, jungle bunny, junglebunny, junkie, junky, kafir, kawk, kike, kikes, kill, kinbaku, kinkster, kinky, kkk, klan, knob, knob end, knobbing, knobead, knobed, knobend, knobhead, knobjocky, knobjokey, kock, kondum, kondums, kooch, kooches, kootch, kraut, kum, kummer, kumming, kums, kunilingus, kunja, kunt, kwif, kyke, l3i+ch, l3itch, labia, lameass, lardass, leather restraint, leather straight jacket, lech, lemon party, leper, lesbian, lesbians, lesbo, lesbos, lez, lezbian, lezbians, lezbo, lezbos, lezza, lezzie, lezzies, lezzy, lmao, lmfao, loin, loins, lolita, looney, lovemaking, lube, lust, lusting, lusty, m0f0, m0fo, m45terbate, ma5terb8, ma5terbate, mafugly, make me come, male squirting, mams, masochist, massa, masterb8, masterbat, masterbat3, masterbate, master-bate, masterbating, masterbation, masterbations, masturbate, masturbating, masturbation, maxi, mcfagget, menage a trois, menses, menstruate, menstruation, meth, m-fucking, mick, middle finger, midget, milf, minge, minger, missionary position, mof0, mofo, mo-fo, molest, mong, moo moo foo foo, moolie, moron, mothafuck, mothafucka, mothafuckas, mothafuckaz, mothafucked, mothafucker, mothafuckers, mothafuckin, mothafucking, mothafuckings, mothafucks, mother fucker, motherfuck, motherfucka, motherfucked, motherfucker, motherfuckers, motherfuckin, motherfucking, motherfuckings, motherfuckka, motherfucks, mound of venus, mr hands, mtherfucker, mthrfucker, mthrfucking, muff, muff diver, muff puff, muffdiver, muffdiving, munging, munter, murder, mutha, muthafecker, muthafuckaz, muthafuckker, muther, mutherfucker, mutherfucking, muthrfucking, n1gga, n1gger, nad, nads, naked, nambla, napalm, nappy, nawashi, nazi, nazism, need the dick, negro, neonazi, nig nog, nigaboo, nigg3r, nigg4h, nigga, niggah, niggas, niggaz, nigger, niggers, niggle, niglet, nig-nog, nimphomania, nimrod, ninny, nipple, nipples, nob, nob jokey, nobhead, nobjocky, nobjokey, nonce, nooky, nsfw images, nude, nudity, numbnuts, nut butter, nut sack, nutsack, nutter, nympho, nymphomania, octopussy, old bag, omg, omorashi, one cup two girls, one guy one jar, opiate, opium, oral, orally, organ, orgasim, orgasims, orgasm, orgasmic, orgasms, orgies, orgy, ovary, ovum, ovums, p.u.s.s.y., p0rn, paddy, paedophile, paki, panooch, pansy, pantie, panties, panty, pastie, pasty, pawn, pcp, pecker, peckerhead, pedo, pedobear, pedophile, pedophilia, pedophiliac, pee, peepee, pegging, penetrate, penetration, penial, penile, penis, penisbanger, penisfucker, penispuffer, perversion, peyote, phalli, phallic, phone sex, phonesex, phuck, phuk, phuked, phuking, phukked, phukking, phuks, phuq, piece of shit, pigfucker, pikey, pillowbiter, pimp, pimpis, pinko, piss, piss off, piss pig, pissed, pissed off, pisser, pissers, pisses, pissflaps, pissin, pissing, pissoff, piss-off, pisspig, playboy, pleasure chest, pms, polack, pole smoker, polesmoker, pollock, ponyplay, poof, poon, poonani, poonany, poontang, poop, poop chute, poopchute, poopuncher, porch monkey, porchmonkey, porn, porno, pornography, pornos, pot, potty, prick, pricks, prickteaser, prig, prince albert piercing, prod, pron, prostitute, prude, psycho, pthc, pube, pubes, pubic, pubis, punani, punanny, punany, punkass, punky, punta, puss, pusse, pussi, pussies, pussy, pussy fart, pussy palace, pussylicking, pussypounder, pussys, pust, puto, queaf, queef, queer, queerbait, queerhole, queero, queers, quicky, quim, racy, raghead, raging boner, rape, raped, raper, rapey, raping, rapist, raunch, rectal, rectum, rectus, reefer, reetard, reich, renob, retard, retarded, reverse cowgirl, revue, rimjaw, rimjob, rimming, ritard, rosy palm, rosy palm and her 5 sisters, rtard, r-tard, rubbish, rum, rump, rumprammer, ruski, rusty trombone, s hit, s&m, s.h.i.t., s.o.b., s_h_i_t, s0b, sadism, sadist, sambo, sand nigger, sandbar, sandler, sandnigger, sanger, santorum, sausage queen, scag, scantily, scat, schizo, schlong, scissoring, screw, screwed, screwing, scroat, scrog, scrot, scrote, scrotum, scrud, scum, seaman, seamen, seduce, seks, semen, sex, sexo, sexual, sexy, sh!+, sh!t, sh1t, s-h-1-t, shag, shagger, shaggin, shagging, shamedame, shaved beaver, shaved pussy, shemale, shi+, shibari, shirt lifter, shit, s-h-i-t, shit ass, shit fucker, shitass, shitbag, shitbagger, shitblimp, shitbrains, shitbreath, shitcanned, shitcunt, shitdick, shite, shiteater, shited, shitey, shitface, shitfaced, shitfuck, shitfull, shithead, shitheads, shithole, shithouse, shiting, shitings, shits, shitspitter, shitstain, shitt, shitted, shitter, shitters, shittier, shittiest, shitting, shittings, shitty, shiz, shiznit, shota, shrimping, sissy, skag, skank, skeet, skullfuck, slag, slanteye, slave, sleaze, sleazy, slope, slut, slut bucket, slutbag, slutdumper, slutkiss, sluts, smartass, smartasses, smeg, smegma, smut, smutty, snatch, sniper, snowballing, snuff, s-o-b, sod off, sodom, sodomize, sodomy, son of a bitch, son of a motherless goat, son of a whore, son-of-a-bitch, souse, soused, spac, spade, sperm, spic, spick, spik, spiks, splooge, splooge moose, spooge, spook, spread legs, spunk, steamy, stfu, stiffy, stoned, strap on, strapon, strappado, strip, strip club, stroke, stupid, style doggy, suck, suckass, sucked, sucking, sucks, suicide girls, sultry women, sumofabiatch, swastika, swinger, t1t, t1tt1e5, t1tties, taff, taig, tainted love, taking the piss, tampon, tard, tart, taste my, tawdry, tea bagging, teabagging, teat, teets, teez, terd, teste, testee, testes, testical, testicle, testis, threesome, throating, thrust, thug, thundercunt, tied up, tight white, tinkle, tit, tit wank, titfuck, titi, tities, tits, titt, tittie5, tittiefucker, titties, titty, tittyfuck, tittyfucker, tittywank, titwank, toke, tongue in a, toots, topless, tosser, towelhead, tramp, tranny, transsexual, trashy, tribadism, trumped, tub girl, tubgirl, turd, tush, tushy, tw4t, twat, twathead, twatlips, twats, twatty, twatwaffle, twink, twinkie, two fingers, two fingers with tongue, two girls one cup, twunt, twunter, ugly, unclefucker, undies, undressing, unwed, upskirt, urethra play, urinal, urine, urophilia, uterus, uzi, v14gra, v1gra, vag, vagina, vajayjay, va-j-j, valium, venus mound, veqtable, viagra, vibrator, violet wand, virgin, vixen, vjayjay, vodka, vomit, vorarephilia, voyeur, vulgar, vulva, w00se, wad, wang, wank, wanker, wankjob, wanky, wazoo, wedgie, weed, weenie, weewee, weiner, weirdo, wench, wet dream, wetback, wh0re, wh0reface, white power, whitey, whiz, whoar, whoralicious, whore, whorealicious, whorebag, whored, whoreface, whorehopper, whorehouse, whores, whoring, wigger, willies, willy, window licker, wiseass, wiseasses, wog, womb, woody, wop, wrapping men, wrinkled starfish, wtf, xrated, x-rated, xx, xxx, yaoi, yeasty, yellow showers, yid, yiffy, yobbo, zoophile, zoophilia, zubb
"""

In [40]:
with open('/content/drive/MyDrive/IRTM/profanity.txt', 'w') as file:
  file.write(profanities_raw)
  file.close()


In [42]:
profanities = []
with open('/content/drive/MyDrive/IRTM/profanity.txt', 'r') as file:
    prof_string = file.read().replace('\n', '')
    prof_tokens = prof_string.split(", ")
    for token in prof_tokens:
        profanities.append(token)

In [43]:
for s,song in tqdm(enumerate(lyric_corpus_tokenized)):
    filtered_text = []    
    for token in song:
        if token not in profanities:
            filtered_text.append(token)
    lyric_corpus_tokenized[s] = filtered_text

760it [00:10, 69.16it/s]


In [44]:
count_token_in_corpus(lyric_corpus_tokenized)

Total of 208276 tokens in the current corpus.


In [45]:
# optionally store tokenized lyrics 
fileObject = open("/content/drive/MyDrive/IRTM/lyric_corpus_tokenized",'wb') 
pickle.dump(lyric_corpus_tokenized,fileObject)   
fileObject.close()

In [46]:
file_path  = open('/content/drive/MyDrive/IRTM/lyric_corpus_tokenized','rb')
lyric_corpus_tokenized = pickle.load(file_path)
file_path.close()

In [47]:
from gensim.corpora import Dictionary

dictionary = Dictionary(lyric_corpus_tokenized)
print('Number of unique tokens: ', len(dictionary))

Number of unique tokens:  8888


In [48]:
dictionary.filter_extremes(no_below = 100, no_above = 0.8)
print('Number of unique tokens: ', len(dictionary))

Number of unique tokens:  244


In [49]:
from gensim.corpora import MmCorpus

gensim_corpus = [dictionary.doc2bow(song) for song in lyric_corpus_tokenized]

#create index to dictionary
temp = dictionary[0]  # "loads" the dictionary
id2word = dictionary.id2token

In [50]:
# training parameters
chunksize = 2000
passes = 20
iterations = 100
num_topics = 6

In [51]:
from gensim.models import LdaModel
lda_model = LdaModel(
corpus=gensim_corpus,
id2word=id2word,
chunksize=chunksize,
alpha='auto',
eta='auto',
iterations=iterations,
num_topics=num_topics,
passes=passes
)

In [52]:
from gensim.models.coherencemodel import CoherenceModel

coherencemodel = CoherenceModel(model=lda_model, texts=lyric_corpus_tokenized, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

0.2808123446170799


In [53]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
pyLDAvis.display(vis_data)


In [54]:
pyLDAvis.save_html(vis_data, './Lyrics_LDA_k_'+ str(num_topics) +'.html')